In [51]:
from google.cloud import bigquery
project_id="students-group2"
client = bigquery.Client(project=project_id)

In [52]:
client.list_tables("students-group2.fatma_aziz")

In [53]:
# les noms des colonnes de la table movies 
movies = client.get_table("students-group2.fatma_aziz.movies")
print([schema_field.name for schema_field in movies.schema])

['movieId', 'title', 'genres']


In [54]:
# le nom des colonnes de la table ratings 
ratings = client.get_table("students-group2.fatma_aziz.ratings")
print([schema_field.name for schema_field in ratings.schema])

['userId', 'movieId', 'rating', 'timestamp']


In [55]:
# Dataframes 
df_movies = client.query("SELECT movieId, title, genres FROM `students-group2.fatma_aziz.movies`").to_dataframe()
df_ratings = client.query("SELECT userId, movieId, rating, timestamp FROM `students-group2.fatma_aziz.ratings`").to_dataframe()

In [56]:
df_movies

,movieId,title,genres
0,126929,Li'l Quinquin ( ),(no genres listed)
1,135460,Pablo (2012),(no genres listed)
2,138863,The Big Broadcast of 1936 (1935),(no genres listed)
3,141305,Round Trip to Heaven (1992),(no genres listed)
4,141472,The 50 Year Argument (2014),(no genres listed)
...,...,...,...
10324,85896,Tribute to a Bad Man (1956),Western
10325,103570,Dead Man's Burden (2012),Western
10326,105223,Colorado Territory (1949),Western
10327,128360,The Hateful Eight (2015),Western


In [57]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  Int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: Int64(1), object(2)
memory usage: 252.3+ KB


In [58]:
df_ratings

,userId,movieId,rating,timestamp
0,1,204,0.5,1217895786
1,1,256,0.5,1217895764
2,1,277,0.5,1217895772
3,1,719,0.5,1217895799
4,1,45950,0.5,1217897813
...,...,...,...,...
105334,668,93040,5.0,1331051757
105335,668,98154,5.0,1353208964
105336,668,101862,5.0,1373168467
105337,668,106916,5.0,1388740601


In [59]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  Int64  
 1   movieId    105339 non-null  Int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  Int64  
dtypes: Int64(3), float64(1)
memory usage: 3.5 MB


In [61]:
print("le nombre de films total:",len(df_movies['movieId'].value_counts()))

le nombre de films total: 10329


In [62]:
print("le nombre de films ayant des ratings:",len(df_ratings['movieId'].value_counts()))

le nombre de films ayant des ratings: 10325


In [63]:
print("le nombre d'utilisateurs:",len(df_ratings['userId'].value_counts()))

le nombre d'utilisateurs: 668


In [64]:
print("les valeurs possibles de rating:", df_ratings['rating'].value_counts())

les valeurs possibles de rating: rating
4.0    28880
3.0    21729
5.0    14856
3.5    12237
4.5     8187
2.0     7943
2.5     5484
1.0     3258
1.5     1567
0.5     1198
Name: count, dtype: int64


In [65]:
print("les genres de films:", df_movies['genres'].value_counts())

les genres de films: genres
Drama                                               1385
Comedy                                               826
Comedy|Drama                                         465
Drama|Romance                                        421
Comedy|Romance                                       363
                                                    ... 
Adventure|Romance|War|Western                          1
Adventure|Romance|Western                              1
Adventure|Thriller|Western                             1
Action|Adventure|Thriller|War                          1
Adventure|Fantasy|Horror|Romance|Sci-Fi|Thriller       1
Name: count, Length: 938, dtype: int64


In [66]:
# Essayer des requetes BigQuery 
avg_rating_per_movie = client.query("""
    SELECT movieId, AVG(rating) AS avg_rating, COUNT(*) AS n_ratings
    FROM `students-group2.fatma_aziz.ratings`
    GROUP BY movieId
    ORDER BY avg_rating DESC
""").to_dataframe()

avg_rating_per_movie

,movieId,avg_rating,n_ratings
0,138702,5.0,1
1,4475,5.0,1
2,3914,5.0,1
3,1471,5.0,1
4,100527,5.0,1
...,...,...,...
10320,61123,0.5,1
10321,2449,0.5,1
10322,50147,0.5,1
10323,66472,0.5,1


In [67]:
movie_avg_ratings = client.query("""
    SELECT m.movieId, m.title, AVG(r.rating) AS avg_rating
    FROM `students-group2.fatma_aziz.ratings` r
    JOIN `students-group2.fatma_aziz.movies` m
    ON r.movieId = m.movieId
    GROUP BY m.movieId, m.title
    ORDER BY avg_rating DESC
""").to_dataframe()

In [69]:
movie_avg_ratings

,movieId,title,avg_rating
0,3914,"Broken Hearts Club, The (2000)",5.0
1,716,Switchblade Sisters (1975),5.0
2,102666,Ivan Vasilievich: Back to the Future (Ivan Vas...,5.0
3,32551,Speedy (1928),5.0
4,1546,Schizopolis (1996),5.0
...,...,...,...
10320,6569,Hotel (2001),0.5
10321,33629,Airborne (1993),0.5
10322,53282,Day Night Day Night (2006),0.5
10323,7259,You Got Served (2004),0.5


In [68]:
# Nombre de rating par utilisateur
ratings_per_user = client.query("""
    SELECT userId, COUNT(*) AS n_ratings
    FROM `students-group2.fatma_aziz.ratings`
    GROUP BY userId
    ORDER BY n_ratings DESC
""").to_dataframe()


In [70]:
ratings_per_user

,userId,n_ratings
0,668,5678
1,575,2837
2,458,2086
3,232,1421
4,310,1287
...,...,...
663,58,20
664,48,20
665,654,20
666,517,20


In [71]:
df_movies_no_ratings = client.query("""
SELECT 
  m.movieId,
  m.title,
  m.genres
FROM `students-group2.fatma_aziz.movies` m
LEFT JOIN `students-group2.fatma_aziz.ratings` r
  ON m.movieId = r.movieId
WHERE r.movieId IS NULL
"""
).to_dataframe()

print("Nombre de films sans ratings :", df_movies_no_ratings.shape[0])
df_movies_no_ratings.head()

Nombre de films sans ratings : 4


,movieId,title,genres
0,128991,Johnny Express (2014),Animation|Comedy|Sci-Fi
1,54290,Bratz: The Movie (2007),Comedy
2,7243,Intolerance: Love's Struggle Throughout the Ag...,Drama
3,8785,Early Summer (Bakushû) (1951),Drama


In [75]:
df_number_movie_per_genre = client.query("""
SELECT 
  genres,
  COUNT(*) AS number_of_movies
FROM `students-group2.fatma_aziz.movies`
GROUP BY genres
ORDER BY number_of_movies DESC
""").to_dataframe()

print("Number of movies per genres")
df_number_movie_per_genre.head(10)


Number of movies per genres


,genres,number_of_movies
0,Drama,1385
1,Comedy,826
2,Comedy|Drama,465
3,Drama|Romance,421
4,Comedy|Romance,363
5,Documentary,300
6,Comedy|Drama|Romance,286
7,Drama|Thriller,212
8,Crime|Drama,188
9,Horror,170
